In [106]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from plotly import graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly
plotly.tools.set_credentials_file(username='matiscakosky', api_key='6tANyDngJzUP1YhgYHrr')


In [101]:
df = pd.read_csv('events.csv',low_memory=False)

In [102]:
visited = df.loc[df.event == 'visited site'].loc[df['new_vs_returning']=='New']
viewed = df.loc[df['event'] =='viewed product']
checkout = df.loc[df['event']=='checkout']
conversion = df.loc[df['event']=='conversion']

In [103]:
funnel = visited['person'].to_frame()
funnel = funnel.set_index('person')
funnel['visitas']=visited['person'].value_counts()
funnel['vioProducto']=viewed['person'].value_counts()
funnel['hizoCheckout']=checkout['person'].value_counts()
funnel['compro']= conversion['person'].value_counts()
funnel['compro']= funnel['compro'].fillna(0)
funnel.head()

,visitas,vioProducto,hizoCheckout,compro
person,,,,
0004b0a2,1,1.0,1,0.0
0006a21a,1,1.0,1,0.0
000a54b2,1,19.0,1,0.0
00184bf9,1,36.0,4,0.0
0019c395,1,2.0,1,0.0


In [104]:
cantidadDeVisitas = funnel.visitas.sum()
cantidadDeProductosVisitados = funnel.vioProducto.sum()
cantidadDeCheckout = funnel.hizoCheckout.sum()
cantidadDeCompras = funnel.compro.sum()


In [117]:

data_table = [['Evento', 'Cantidad de veces realizado'],
               ['Visitas', cantidadDeVisitas],
               ['Visitados', cantidadDeProductosVisitados],
               ['Checkout', cantidadDeCheckout],
               ['Purchase', cantidadDeCompras]]

table = ff.create_table(data_table)
py.iplot(table)

In [118]:
# chart stages data
values = [cantidadDeVisitas, cantidadDeProductosVisitados, cantidadDeCheckout, cantidadDeCompras]
phases = ['Visitas', 'ViewedProduct', 'Checkout', 'Purchase']

# color of each funnel section
colors = ['rgb(32,155,160)', 'rgb(253,93,124)', 'rgb(28,119,139)', 'rgb(182,231,235)', 'rgb(35,154,160)']

n_phase = len(phases)
plot_width = 400

# height of a section and difference between sections 
section_h = 100
section_d = 10

# multiplication factor to calculate the width of other sections
unit_width = plot_width / max(values)

# width of each funnel section relative to the plot width
phase_w = [int(value * unit_width) for value in values]

# plot height based on the number of sections and the gap in between them
height = section_h * n_phase + section_d * (n_phase - 1)

In [119]:
# list containing all the plot shapes
shapes = []

# list containing the Y-axis location for each section's name and value text
label_y = []

for i in range(n_phase):
        if (i == n_phase-1):
                points = [phase_w[i] / 2, height, phase_w[i] / 2, height - section_h]
        else:
                points = [phase_w[i] / 2, height, phase_w[i+1] / 2, height - section_h]

        path = 'M {0} {1} L {2} {3} L -{2} {3} L -{0} {1} Z'.format(*points)

        shape = {
                'type': 'path',
                'path': path,
                'fillcolor': colors[i],
                'line': {
                    'width': 1,
                    'color': colors[i]
                }
        }
        shapes.append(shape)
        
        # Y-axis location for this section's details (text)
        label_y.append(height - (section_h / 2))

        height = height - (section_h + section_d)

In [120]:
# For phase names
label_trace = go.Scatter(
    x=[-350]*n_phase,
    y=label_y,
    mode='text',
    text=phases,
    textfont=dict(
        color='rgb(200,200,200)',
        size=15
    )
)
 
# For phase values
value_trace = go.Scatter(
    x=[350]*n_phase,
    y=label_y,
    mode='text',
    text=values,
    textfont=dict(
        color='rgb(200,200,200)',
        size=15
    )
)

data = [label_trace, value_trace]
 
layout = go.Layout(
    title="<b>Flujo de compras</b>",
    titlefont=dict(
        size=20,
        color='rgb(203,203,203)'
    ),
    shapes=shapes,
    height=560,
    width=800,
    showlegend=False,
    paper_bgcolor='rgba(44,58,71,1)',
    plot_bgcolor='rgba(44,58,71,1)',
    xaxis=dict(
        showticklabels=False,
        zeroline=False,
    ),
    yaxis=dict(
        showticklabels=False,
        zeroline=False
    )
)
 
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)